In [39]:
# CMPS140 Project - How to Become Top Earner in Airbnb?
# Tangni Wang, Tung Hoi Man, Yunxiang Fu
# {twang63, tuman, yfu7}@ucsc.edu

In [ ]:
# To do:

# Split Training set, testing set in code?
# Convert attributes to numbers
# Generate graph
# Different model with different functions? Which function do we use?


In [40]:
# Helper functions
def to_int(x):
    '''
    Columns value corrections
    '''
    if x=='f':
        x=x.replace('f','0')
    elif x=='t':
        x=x.replace('t','1')
    else:
        x= '0'
    return int(x)

def changeTime(x):
    '''
    change host_response_time columns from string into numerical.
    '''
    if x == 'within an hour':
        x='1'
    elif x == 'within a few hours':
        x='4'
    elif x == 'within a day':
        x='24'
    elif x == 'a few days or more':
        x='48'
    else:
        x='96'
        
    return x


def changeStr(x):
    '''
    change back the host_response_time from the numerical into strings
    '''
    if x == 1:
        x='within an hour'
    elif x == 4:
        x='within a few hours'
    elif x == 24:
        x='within a day'
    elif x == 48:
        x= 'a few days or more'
    elif x == 96:
        x= 'Not Response'
        
    return x

In [41]:
# Parsing csv file
import pandas as pd

seattle_file_path = 'seattle_data/training.csv'
seattle_data = pd.read_csv(seattle_file_path, encoding = 'utf-8')
seattle_data.columns

Index(['id', 'listing_url', 'scrape_id', 'last_scraped', 'name', 'summary',
       'space', 'description', 'experiences_offered', 'neighborhood_overview',
       'notes', 'transit', 'thumbnail_url', 'medium_url', 'picture_url',
       'xl_picture_url', 'host_id', 'host_url', 'host_name', 'host_since',
       'host_location', 'host_about', 'host_response_time',
       'host_response_rate', 'host_acceptance_rate', 'host_is_superhost',
       'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood',
       'host_listings_count', 'host_total_listings_count',
       'host_verifications', 'host_has_profile_pic', 'host_identity_verified',
       'street', 'neighbourhood', 'neighbourhood_cleansed',
       'neighbourhood_group_cleansed', 'city', 'state', 'zipcode', 'market',
       'smart_location', 'country_code', 'country', 'latitude', 'longitude',
       'is_location_exact', 'property_type', 'room_type', 'accommodates',
       'bathrooms', 'bedrooms', 'beds', 'bed_type', 'amenities', '

In [42]:
# Baseline model
# Need to follow some examples to do it properly.


In [43]:
# Make a new attribute
seattle_data_new = seattle_data.copy()
seattle_data_new['new_score_reviews2'] = seattle_data['reviews_per_month'] * seattle_data['review_scores_rating'] / 10
# Fill missing value with zero for column
seattle_data_new['new_score_reviews2'].fillna(0, inplace = True)
seattle_data_new.head()

id                           listing_url     scrape_id last_scraped  \
0   241032   https://www.airbnb.com/rooms/241032  2.020000e+13       1/4/16   
1   953595   https://www.airbnb.com/rooms/953595  2.020000e+13       1/4/16   
2  3308979  https://www.airbnb.com/rooms/3308979  2.020000e+13       1/4/16   
3  7421966  https://www.airbnb.com/rooms/7421966  2.020000e+13       1/4/16   
4   278830   https://www.airbnb.com/rooms/278830  2.020000e+13       1/4/16   

                                  name  \
0         Stylish Queen Anne Apartment   
1   Bright & Airy Queen Anne Apartment   
2  New Modern House-Amazing water view   
3                   Queen Anne Chateau   
4       Charming craftsman 3 bdm house   

                                             summary  \
0                                                NaN   
1  Chemically sensitive? We've removed the irrita...   
2  New modern house built in 2013.  Spectacular s...   
3  A charming apartment that sits atop Queen Anne...   
4  Cozy family craftman house in beautiful neighb...   

                                               space  \
0  Make your self at home in this charming one-be...   
1  Beautiful, hypoallergenic apartment in an extr...   
2  Our house is modern, light and fresh with a wa...   
3                                                NaN   
4  Cozy family craftman house in beautiful neighb...   

                                         description experiences_offered  \
0  Make your self at home in this charming one-be...                none   
1  Chemically sensitive? We've removed the irrita...                none   
2  New modern house built in 2013.  Spectacular s...                none   
3  A charming apartment that sits atop Queen Anne...                none   
4  Cozy family craftman house in beautiful neighb...                none   

                               neighborhood_overview        ...          \
0                                                NaN        ...           
1  Queen Anne is a wonderful, truly functional vi...        ...           
2  Upper Queen Anne is a charming neighborhood fu...        ...           
3                                                NaN        ...           
4  We are in the beautiful neighborhood of Queen ...        ...           

  requires_license license jurisdiction_names instant_bookable  \
0                f     NaN         WASHINGTON                f   
1                f     NaN         WASHINGTON                f   
2                f     NaN         WASHINGTON                f   
3                f     NaN         WASHINGTON                f   
4                f     NaN         WASHINGTON                f   

  cancellation_policy require_guest_profile_picture  \
0            moderate                             f   
1              strict                             t   
2              strict                             f   
3            flexible                             f   
4              strict                             f   

   require_guest_phone_verification calculated_host_listings_count  \
0                                 f                              2   
1                                 t                              6   
2                                 f                              2   
3                                 f                              1   
4                                 f                              1   

  reviews_per_month new_score_reviews2  
0              4.07             38.665  
1              1.48             14.208  
2              1.15             11.155  
3               NaN              0.000  
4              0.89              8.188  

[5 rows x 93 columns]

In [44]:
# seattle_data_new['new_score_reviews2'].describe()

In [45]:
# imputation fills in the missing value with some number.
# The imputed value won't be exactly right in most cases, 
# but it usually gives more accurate models than dropping the column entirely.

# from sklearn.impute import SimpleImputer
# my_imputer = SimpleImputer()
# seattle_data_with_imputed_values = my_imputer.fit_transform(seattle_data)

# Fill missing data
# seattle_data = seattle_data.fillna(" ")

# Prediction target
# New approach
# estimated_income = (availability_30 * price * 0.2) + (review_per_month * review_scores_rating * 0.8)
# y = earning_potential
# y = (seattle_data.availability_30 * seattle_data.price * 0.1) + (seattle_data.review_per_month * seattle_data.review_scores_rating * 0.9)

# Original approach: new_score_reviews2
y = seattle_data_new.new_score_reviews2


In [51]:
# Fill missing value with zero for numeric type attributes
seattle_data_new['price'].fillna(0, inplace = True)
seattle_data_new['cleaning_fee'].fillna(0, inplace = True)
seattle_data_new['minimum_nights'].fillna(1, inplace = True)
X.head()

# Convert string to number
# Some attribute like host_response_time has string values: 'within a few hours', 'within an hour', 'within a day'...


price  cleaning_fee  minimum_nights
0     85           0.0               1
1    150          40.0               2
2    975         300.0               4
3    100           0.0               1
4    450         125.0               1

In [47]:
# Features
# Needed for sure: 
# cleaning_fee, host_response_time, host_is_superhost, minimum_nights, instant_bookable, cancellation_policy
seattle_features = [
    'price', 
    'cleaning_fee', 
#     'host_response_time', 
#     'host_is_superhost',
    'minimum_nights',
#     'availability_30', 
#     'review_scores_rating',
#     'instant_bookable',
#     'cancellation_policy',
#     'reviews_per_month',
#     'new_score_reviews2',
 ]
X = seattle_data_new[seattle_features]

# Statistic of the features
X.describe()

price  cleaning_fee  minimum_nights
count  2443.000000   2443.000000     2443.000000
mean    130.103152     48.394597        2.101924
std      90.838342     50.986282        2.168158
min      20.000000      0.000000        1.000000
25%      75.000000      5.000000        1.000000
50%     103.000000     35.000000        2.000000
75%     150.000000     75.000000        2.000000
max     999.000000    300.000000       31.000000

In [48]:
X.head()

price  cleaning_fee  minimum_nights
0     85           0.0               1
1    150          40.0               2
2    975         300.0               4
3    100           0.0               1
4    450         125.0               1

In [52]:
# Making a model
from sklearn.tree import DecisionTreeRegressor
seattle_model = DecisionTreeRegressor(random_state=1)
seattle_model.fit(X,y);

print(seattle_model)

DecisionTreeRegressor(criterion='mse', max_depth=None, max_features=None,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=1, splitter='best')


In [ ]:
# Making a prediction using testing set.
seattle_file_path = 'seattle_data/testing.csv'
seattle_test_data = pd.read_csv(seattle_file_path, encoding = 'utf-8')

# # Prediction target
y2 = seattle_data_new.new_score_reviews2

# Features
melb_test_features = ['Rooms', 'Bathroom', 'Landsize', 'Lattitude', 'Longtitude']
X2 = melb_test[melb_test_features]

print("Making predicitions for the following 5 houses:")
print(X2)
print("The predictions are")
print(melbourne_model.predict(X2))